In [8]:
import os
import platform
import pandas as pd
import numpy as np
import time
import asyncio
import threading
import json
import itertools
import nest_asyncio
nest_asyncio.apply()
from bs4 import BeautifulSoup

# for logging
import sys
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler

from urllib.request import urlopen
from urllib.parse import urlparse, parse_qs

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

#logging
class PrintLogger:
    def __init__(self, log):
        self.terminal = sys.stdout
        self.log = log

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        pass

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
def setup_logging():
    log_formatter = logging.Formatter("%(asctime)s - %(message)s", "%Y-%m-%d %H:%M:%S")
    log_file = f'verval_scrape_{current_date}.log'
    log_handler = TimedRotatingFileHandler(log_file, when="midnight", interval=1, backupCount=30, utc=False)
    log_handler.setFormatter(log_formatter)
    log_handler.setLevel(logging.DEBUG)
    
    logger = logging.getLogger()
    logger.addHandler(log_handler)

    sys.stdout = PrintLogger(log_handler.stream)
setup_logging()

os_system = platform.system()
print('OS SYSTEM:   ', os_system)

#cpu count
num_threads = os.cpu_count()
print(f'Num Threads:    {num_threads}')

# set path ke file chromedriver to operate the Chrome browser.
chrome_version = 'v119_0_6045_105'
if os_system == 'Windows':
    chrome_path = os.path.join('webdriver', 'chrome', os_system, chrome_version, 'chromedriver.exe')
elif os_system == 'Linux':
    chrome_path = os.path.join('webdriver', 'chrome', os_system, chrome_version, 'chromedriver')
else:
    chrome_path = os.path.join('webdriver', 'chrome', 'MacOS', chrome_version, 'chromedriver')

print('CHROME PATH:    ', chrome_path)
#webdriver options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
#overcome limited resource problems
# chrome_options.add_argument('--disable-dev-shm-usage')
#open Browser in maximized mode
chrome_options.add_argument("start-maximized")
#disable extension
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

def driversetup():
  # webdriver_service = ChromeService(ChromeDriverManager().install())
  chrome_service = Service(executable_path=chrome_path)
  driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
  return driver
driver = driversetup()

#create empty dataframe
df_rekapitulasi = df_provinsi = df_kabupaten = df_kecamatan = df_sekolah_id_nama = pd.DataFrame()
kode_wilayah_kabupaten = pd.Series([]) 
# kode_wilayah_kecamatan = pd.Series([])
kode_wilayah_kecamatan = []
sekolah_id_enkrip = pd.Series([])
semester_label = pd.Series([])
semester_period = pd.Series([])
max_retries = 3

province_api = 'https://dapo.kemdikbud.go.id/rekap/dataPD?id_level_wilayah=0&kode_wilayah=000000&semester_id=20231'
provinces = pd.read_json(province_api)

provinces = provinces.sort_values('kode_wilayah')
provinces = provinces['kode_wilayah']
provinces = provinces.astype(str).str.zfill(6)

provinces
"""## get kode wilayah `kabupaten`"""
start_kab_kota = time.time()
for province in provinces[:5]:
    province = province.strip()
    kab_api = 'https://dapo.kemdikbud.go.id/rekap/dataPD?id_level_wilayah=1&kode_wilayah={0}&semester_id=20231'.format(province)

    kabupatens = pd.read_json(kab_api)
    kabupatens = kabupatens.sort_values('kode_wilayah')
    kabupatens = kabupatens['kode_wilayah']
    kode_wilayah_kabupaten = kode_wilayah_kabupaten._append(kabupatens, ignore_index=True)

kode_wilayah_kabupaten = kode_wilayah_kabupaten.astype(str).str.zfill(6)
df_kode_wilayah_kabupaten = pd.DataFrame({'kode_wilayah_kabupaten': kode_wilayah_kabupaten})
df_kode_wilayah_kabupaten.to_csv('./dataset/kode_wilayah_kabupaten_testest.csv', index=False)
print('kode_wilayah_kabupaten: ', len(kode_wilayah_kabupaten))
print(f'get kab/kota done in: {time.time() - start_kab_kota} seconds')


C:\Users\steva\AppData\Local\Temp\ipykernel_9940\2078481630.py:94: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  kode_wilayah_kabupaten = pd.Series([])
C:\Users\steva\AppData\Local\Temp\ipykernel_9940\2078481630.py:97: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sekolah_id_enkrip = pd.Series([])
C:\Users\steva\AppData\Local\Temp\ipykernel_9940\2078481630.py:98: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  semester_label = pd.Series([])
C:\Users\steva\AppData\Local\Temp\ipykernel_9940\2078481630.py:99: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to 

,nama,kode_wilayah,id_level_wilayah,mst_kode_wilayah,induk_provinsi,kode_wilayah_induk_provinsi,induk_kabupaten,kode_wilayah_induk_kabupaten,tk_pd_laki,tk_pd_perempuan,...,slb_pd,sd_pd_laki,sd_pd_perempuan,sd_pd,smp_pd_laki,smp_pd_perempuan,smp_pd,total_pd_laki,total_pd_perempuan,total_pd
0,Prov. Jawa Barat,20000,1,0,NaN,NaN,NaN,NaN,198439,186961,...,28456,2318088,2164358,4482446,931519,884688,1816207,5017173,4677265,9694438
1,Prov. Jawa Timur,50000,1,0,NaN,NaN,NaN,NaN,396283,373403,...,22040,1311888,1217687,2529575,643740,586638,1230378,3416685,3133268,6549953
2,Prov. Jawa Tengah,30000,1,0,NaN,NaN,NaN,NaN,295231,274817,...,20951,1344658,1248140,2592798,620491,587701,1208192,3199641,2980193,6179834
3,Prov. Sumatera Utara,70000,1,0,NaN,NaN,NaN,NaN,65021,61060,...,5732,791398,731284,1522682,318367,303587,621954,1638849,1531695,3170544
4,Prov. Banten,280000,1,0,NaN,NaN,NaN,NaN,53967,51659,...,6666,623427,579309,1202736,227733,214256,441989,1265419,1175334,2440753
5,Prov. Sulawesi Selatan,190000,1,0,NaN,NaN,NaN,NaN,81014,77143,...,4817,456904,424962,881866,167178,163912,331090,965510,913861,1879371
6,Prov. Sumatera Selatan,110000,1,0,NaN,NaN,NaN,NaN,45055,42243,...,3069,457261,423408,880669,180977,175349,356326,936379,874772,1811151
7,Prov. Lampung,120000,1,0,NaN,NaN,NaN,NaN,69298,64940,...,2185,414662,383312,797974,172774,164075,336849,892259,830438,1722697
8,Prov. D.K.I. Jakarta,10000,1,0,NaN,NaN,NaN,NaN,37276,35536,...,6196,390427,365641,756068,176496,171285,347781,862491,814534,1677025
9,Prov. Nusa Tenggara Timur,240000,1,0,NaN,NaN,NaN,NaN,25077,24116,...,3185,348331,314384,662715,161866,154833,316699,769196,724494,1493690


In [15]:
page_url = 'https://dapo.kemdikbud.go.id/pd/3/016006'
page = driver.get(page_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

#get periode for rekapitulasi each semester
semester_dropdown = soup.find(id='selectSemester').find_all('option')[1:]
# semester_dropdown
semester_lists = [semester.get('value') for semester in semester_dropdown]
# semester_lists
semester_lists = [semester_lists[0]]
semester_lists

['20231']

In [8]:
soup.find(id='selectSemester')

In [13]:
rekapitulasi = pd.read_json(link2)
rekapitulasi_list.append(rekapitulasi)

In [16]:
rekapitulasi_list

[                           nama                            sekolah_id  \
 0             TK NEGERI PEMBINA  B69535A3-06CC-49DB-B0D8-5B4B1CF7C591   
 1             KB FADHILUL HAYAT  5EBAE96A-EAEC-4EDA-B643-21EB67458813   
 2                TK ABA ARIDWAN  2FB42DA0-29D7-4681-A2D3-F8D8C2F6005B   
 3                TK AL MUHDIYAT  367FDE02-CA0D-48D4-89C6-44FDB70993C4   
 4                     TK AL NUR  437A5859-E89A-46B2-BE3D-033133D7F3D2   
 ..                          ...                                   ...   
 363           PKBM Mutiara Hati  AC2523DA-07EF-4149-99E1-A9BDBA612658   
 364  PKBM RUMAH SINGGAH TABAYUN  18B00DC5-83D8-4831-98FF-F28C12FDA41E   
 365            SLB NEGERI BOGOR  20F5FCD3-B633-E211-8D4D-0FF45FC8CCE0   
 366            SLB MEKAR SARI 1  00FCD915-2CF5-E011-AF57-3F2D795397EC   
 367              SKB KAB. BOGOR  BCF84686-3F1F-4A28-AD3E-3EC02C16AF14   
 
          npsn  jumlah_kirim  ptk  pegawai   pd  rombel  jml_rk  jml_lab  ...  \
 0    69893404            23 

In [17]:
result = pd.concat(rekapitulasi_list, ignore_index=True)

In [18]:
result

,nama,sekolah_id,npsn,jumlah_kirim,ptk,pegawai,pd,rombel,jml_rk,jml_lab,...,induk_kecamatan,kode_wilayah_induk_kecamatan,induk_kabupaten,kode_wilayah_induk_kabupaten,induk_provinsi,kode_wilayah_induk_provinsi,bentuk_pendidikan,status_sekolah,sinkron_terakhir,sekolah_id_enkrip
0,TK NEGERI PEMBINA,B69535A3-06CC-49DB-B0D8-5B4B1CF7C591,69893404,23,9,5,116,8,7,0,...,Kec. Cibinong,20521,Kab. Bogor,20500,Prov. Jawa Barat,20000,TK,Negeri,30 Oct 2023 11:49:36,4861315EF341EA1FAA4D ...
1,KB FADHILUL HAYAT,5EBAE96A-EAEC-4EDA-B643-21EB67458813,69985297,3,3,1,25,2,2,0,...,Kec. Cibinong,20521,Kab. Bogor,20500,Prov. Jawa Barat,20000,TK,Swasta,27 Aug 2023 10:01:50,55749F1CE52B7B311099 ...
2,TK ABA ARIDWAN,2FB42DA0-29D7-4681-A2D3-F8D8C2F6005B,69908239,10,4,1,40,3,4,0,...,Kec. Cibinong,20521,Kab. Bogor,20500,Prov. Jawa Barat,20000,TK,Swasta,07 Oct 2023 16:20:53,E6C539499173C83EAF78 ...
3,TK AL MUHDIYAT,367FDE02-CA0D-48D4-89C6-44FDB70993C4,69938720,0,0,0,0,0,0,0,...,Kec. Cibinong,20521,Kab. Bogor,20500,Prov. Jawa Barat,20000,TK,Swasta,08 Oct 2022 22:54:06,4B849A7E1F7E71EB2B08 ...
4,TK AL NUR,437A5859-E89A-46B2-BE3D-033133D7F3D2,20268628,20,3,1,42,3,3,0,...,Kec. Cibinong,20521,Kab. Bogor,20500,Prov. Jawa Barat,20000,TK,Swasta,23 Oct 2023 14:20:36,E3E0ED0F56A09D5830A0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,PKBM Pelita Harapan,93E1EEC9-3028-489A-84B6-89A49FC7BEA6,P2960646,11,9,1,83,6,7,1,...,Kec. Cileungsi,20518,Kab. Bogor,20500,Prov. Jawa Barat,20000,PKBM,Swasta,01 Oct 2023 10:53:31,CD4806B70613C6A25713 ...
699,PKBM PEMUDA MUHAMMADIYAH,D8F60EC8-CC2D-4073-A519-3A93FAF20E9A,P9970217,47,1,2,97,7,6,0,...,Kec. Cileungsi,20518,Kab. Bogor,20500,Prov. Jawa Barat,20000,PKBM,Swasta,10 Oct 2023 22:36:06,E329639B2B301DBC1673 ...
700,PKBM Perintis,3D071D6D-4F10-4738-9B9C-CB2D310769A2,P2960647,0,0,0,0,0,0,0,...,Kec. Cileungsi,20518,Kab. Bogor,20500,Prov. Jawa Barat,20000,PKBM,Swasta,03 Apr 2023 11:15:12,229FEF63CA6174FC5DC6 ...
701,PKBM SETYA DHARMA,C5B10EF7-EEBD-4906-94DD-EAD2F6D70CFE,P9997213,61,7,1,424,17,3,0,...,Kec. Cileungsi,20518,Kab. Bogor,20500,Prov. Jawa Barat,20000,PKBM,Swasta,30 Oct 2023 11:17:59,C17A4093438D13977866 ...


In [75]:
"""## get kode wilayah `Kecamatan`"""
start_kecamatan = time.time()
print('getting kecamatan')

def get_kecamatan(kode_wilayah_kabupaten):
    print('in get_kecamatan')
    for kabupaten in kode_wilayah_kabupaten:
        retry_count=0
        while retry_count < max_retries:
            try:
                kabupaten = kabupaten.strip()
                kec_api = f'https://dapo.kemdikbud.go.id/rekap/dataPD?id_level_wilayah=2&kode_wilayah={kabupaten}&semester_id=20231'
                kecamatan = pd.read_json(kec_api)
                kecamatan = kecamatan.sort_values('kode_wilayah')
                kecamatan = kecamatan['kode_wilayah']
                # print(kecamatan)
                # kode_wilayah_kecamatan = kode_wilayah_kecamatan._append(kecamatans, ignore_index=True)
                kode_wilayah_kecamatan.append(kecamatan)
                # print(kode_wilayah_kecamatan)
                break
            except Exception as e:
                print(f'kode kabupaten: {kabupaten}, error: {e}, retry: {retry_count}')
                if retry_count == 2:
                    print(f'kecamatan with kabupaten code {kabupaten} was failed to be get')
                retry_count += 1
                time.sleep(3)
    return None

def main_get_kode_kecamatan(kode_wilayah_kabupaten):
    print('in main_get_kode_kecamatan')
    threads = []
    kabupaten_batches = np.array_split(kode_wilayah_kabupaten, num_threads)
    for t in range(num_threads):
        thread = threading.Thread(target=get_kecamatan, args=(kabupaten_batches[t],))
        threads.append(thread)

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()
    return None


kode_wilayah_kabupaten = pd.read_csv('dataset\kode_wilayah_kabupaten.csv')
kode_wilayah_kabupaten = kode_wilayah_kabupaten['kode_wilayah_kabupaten']
kode_wilayah_kabupaten = kode_wilayah_kabupaten.astype(str).str.zfill(6)
main_get_kode_kecamatan(kode_wilayah_kabupaten)

kode_wilayah_kecamatan = pd.concat(kode_wilayah_kecamatan, ignore_index=True)
kode_wilayah_kecamatan = kode_wilayah_kecamatan.astype(str).str.zfill(6)
df_kode_wilayah_kecamatan = pd.DataFrame({'kode_wilayah_kecamatan': kode_wilayah_kecamatan})
df_kode_wilayah_kecamatan.to_csv('./dataset/kode_wilayah_kecamatan_thread.csv', index=False)

In [77]:
rekapitulasi_urls = []

In [100]:
"""## get semester list"""
#contoh page kecamatan kemayoran. untuk ngambil tag select semester
page_url = 'https://dapo.kemdikbud.go.id/pd/3/016006'
page = driver.get(page_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

#get periode for rekapitulasi each semester
semester_dropdown = soup.find(id='selectSemester').find_all('option')[1:]
semester_lists = [semester.get('value') for semester in semester_dropdown]
semester_lists = [semester_lists[0]]

print('semester lists ', semester_lists)

semester_titles = [semester.get_text().strip() for semester in semester_dropdown]
print('semester_titles ', semester_titles)


"""## generate rekapitulasi urls in kecamatan level"""
start_rekap = time.time()
kode_wilayah_kecamatan = pd.read_csv('dataset\kode_wilayah_kecamatan_thread.csv')
kode_wilayah_kecamatan = kode_wilayah_kecamatan['kode_wilayah_kecamatan']
kode_wilayah_kecamatan = kode_wilayah_kecamatan.astype(str).str.zfill(6)
rekapitulasi_urls = ['https://dapo.kemdikbud.go.id/rekap/progresSP?id_level_wilayah=3&kode_wilayah={0}&semester_id={1}'.format(kode, semester) for kode, semester in itertools.product(kode_wilayah_kecamatan, semester_lists)]
rekapitulasi_urls = pd.DataFrame({'urls': rekapitulasi_urls})
rekapitulasi_urls.to_csv('./dataset/rekapitulasi_urls_thread_20231.csv', index=False)

In [99]:
rekapitulasi_urls

,urls
0,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
1,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
2,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
3,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
4,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
...,...
7254,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
7255,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
7256,https://dapo.kemdikbud.go.id/rekap/progresSP?i...
7257,https://dapo.kemdikbud.go.id/rekap/progresSP?i...


In [103]:
rekapitulasi_result = pd.read_csv('dataset\\rekapitulasi_result_20231.csv')

In [104]:
rekapitulasi_result

,nama,sekolah_id,npsn,jumlah_kirim,ptk,pegawai,pd,rombel,jml_rk,jml_lab,...,before_tempat_bermain_olahraga,after_tempat_bermain_olahraga,before_ruang_tu,after_ruang_tu,before_ruang_konseling,after_ruang_konseling,before_ruang_osis,after_ruang_osis,before_bangunan,after_bangunan
0,TK KI HAJAR DEWANTARA NUAMURI,DF13C321-8426-4599-B17B-88E168645FF3,50305479,5.0,2.0,1.0,48.0,2.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TK MARIA FATIMA KOANARA,901D8769-109B-45DA-85FC-B8095E991DAD,50305704,10.0,1.0,0.0,34.0,2.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TK TANA NUWA,26C4C324-49DA-484E-8151-8D7778779272,50305478,2.0,1.0,1.0,10.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KB MARLOM,8F7307FE-458B-4228-855C-F2D114CE8C0A,70006252,6.0,5.0,0.0,19.0,2.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KB. KELIMUTU PERMAI,8C3439B9-7428-4E24-9FFF-E21F0A92CC99,69962391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437056,SMK NEGERI 2 RAJA AMPAT,4BD7E95D-439E-442D-B327-3BDB88217382,60403788,107.0,33.0,4.0,139.0,15.0,15.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437057,SMK DKV YANURAF KHIDIRRAJA,FA184F6D-0D89-4088-9582-447D41290498,69942220,39.0,11.0,4.0,80.0,5.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437058,SMKS YPK BUKIT ZAITUN WAISAI,2AC74EB5-E8E7-4311-A3F5-04A5FBD867CC,69830685,51.0,32.0,5.0,346.0,13.0,14.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437059,PKBM BAHARI KOFARKOR,905635A6-97F8-40C8-8CD0-DA7BA15A6EFE,P9970082,62.0,2.0,1.0,169.0,8.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
